- 일단 person, forklift, trolly 로만 돌려봄
- 참고 사이트
    https://blog.naver.com/PostView.naver?blogId=beyondlegend&logNo=223050797442

In [16]:
import datetime, cv2, ultralytics
from ultralytics import YOLO

In [17]:
# 커스텀 데이터에 맞는 YAML 파일 만들기 
import yaml

# 저장된 데이터 경로로 변경, 클래스 이름 및 수 변경
data = {'train':"C:/Users/DELL-PC/Desktop/YimSoYoung/labeled/train_ysy",
        'val':"C:/Users/DELL-PC/Desktop/YimSoYoung/labeled/valid_syr",
        'test':"C:/Users/DELL-PC/Desktop/YimSoYoung/labeled/test_lhe",
        'names':['Person', 'Forklift','Trolly'],
        'nc' : 3}

# 지정된 yaml 파일에 작성하기
with open('mnvision.yaml', 'w') as f:
    yaml.dump(data, f)

# 작성된 yaml 파일 확인하기
with open('mnvision.yaml', 'r') as f:
    yolo_yaml = yaml.safe_load(f)
    display(yolo_yaml)

{'names': ['Person', 'Forklift', 'Trolly'],
 'nc': 3,
 'test': 'C:/Users/DELL-PC/Desktop/YimSoYoung/labeled/test_lhe',
 'train': 'C:/Users/DELL-PC/Desktop/YimSoYoung/labeled/train_ysy',
 'val': 'C:/Users/DELL-PC/Desktop/YimSoYoung/labeled/valid_syr'}

In [18]:
# YOLOv8 설치 확인
ultralytics.checks()

Ultralytics YOLOv8.2.16  Python-3.8.19 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Setup complete  (20 CPUs, 31.7 GB RAM, 221.9/943.1 GB disk)


In [19]:
# YOLO 모델 로딩
model = YOLO('yolov8n.pt')  # MS COCO dataset 사전학습된 모델 로딩함 

# 사전학습된 모델에 정의된 클래스 확인 
print(type(model.names), len(model.names))
print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [20]:
import torch
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

True
cuda


In [21]:
import comet_ml

comet_ml.init()

In [22]:
# 커스텀 데이터로 학습하기
# 반드시 yaml을 지정해주어야 한다.
model.train(
    data = 'mnvision.yaml', 
    epochs = 50000, 
    patience = 20, 
    batch = 32, 
    imgsz=720,
    pretrained = False,
    plots = True
    ).to(device)

# patience : 검증 손실이 지정된 에포크 수 도안 개선되 않으면 조기종료 
# imgsz : 입력 이미지의 크기를 (416, 416)으로 조정 

Ultralytics YOLOv8.2.16  Python-3.8.19 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=mnvision.yaml, epochs=50000, time=None, patience=20, batch=32, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train22, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning C:\Users\DELL-PC\Desktop\YimSoYoung\labeled\train_ysy.cache... 239 images, 38 backgrounds, 0 corrupt: 100%|██████████| 277/277 [00:00<?, ?it/s]
val: Scanning C:\Users\DELL-PC\Desktop\YimSoYoung\labeled\valid_syr.cache... 236 images, 40 backgrounds, 0 corrupt: 100%|██████████| 276/276 [00:00<?, ?it/s]


Plotting labels to runs\detect\train22\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 736 train, 736 val
Using 8 dataloader workers
Logging results to runs\detect\train22
Starting training for 50000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    1/50000      6.16G      1.153      4.215       1.37         37        736: 100%|██████████| 9/9 [00:01<00:00,  4.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.07it/s]

                   all        276        236     0.0025      0.676     0.0349     0.0163


...

                   all        276        236      0.682       0.62      0.685      0.442
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 36, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



56 epochs completed in 0.039 hours.
Optimizer stripped from runs\detect\train22\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train22\weights\best.pt, 6.2MB

Validating runs\detect\train22\weights\best.pt...
Ultralytics YOLOv8.2.16  Python-3.8.19 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  4.68it/s]


                   all        276        236      0.924       0.82      0.866      0.592
                Person        276         13      0.865      0.692      0.697      0.326
              Forklift        276        176      0.939      0.852      0.947       0.65
                Trolly        276         47      0.968      0.915      0.954      0.801
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train22
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\DELL-PC\miniconda3\envs\Text_017_38\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\DELL-PC\AppData\Local\Temp\ipykernel_22220\3486389053.py", line 3, in <module>
    model.train(
  File "c:\Users\DELL-PC\miniconda3\envs\Text_017_38\lib\site-packages\ultralytics\utils\__init__.py", line 162, in __getattr__
    raise AttributeError(f"'{name}' object has no attribute '{attr}'. See valid attributes below.\n{self.__doc__}")
AttributeError: 'DetMetrics' object has no attribute 'to'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates wheth

In [ ]:
# 커스텀 데이터로 학습된 모델의 클래스 확인 
print(type(model.names), len(model.names))
print(model.names)

In [ ]:
# 검증하기 
import metrics
model.val()

# 검증 결과 출력 
print(metrics.box.map)    # mAP (mean Average Precision)
print(metrics.box.map50)  # mAP@0.5
print(metrics.box.map75)  # mAP@0.75

In [ ]:
# 커스텀데이터로 학습한 모델 저장하기

# 학습된 모델 불러오기
folder_name = 'train11'
model = YOLO(f'runs/detect/{folder_name}/weights/best.pt')
print('불러오기 완료')

In [ ]:
# 모델 저장하기
model.export(format='onnx', dynamic=True)  # ONNX 포맷으로 저장
print('저장완료')

<hr>

In [ ]:
# 예측하기

# img_file = "01frame_12696.jpg"
# results = model.predict(source = img_file, save=True, conf=0.25)

# source : 예측할 이미지, 비디오 또는 웹캠의 경로나 인덱스를 지정
# save : 예측 결과 저장 여부
# conf : 예측 결과의 신뢰도 임계값을 지정함 

In [ ]:
# 비디오 예측

# video_file = "20231129115714.avi"
# results = model.predict(source= video_file, save=True, conf=0.25)

In [ ]:
# 웹캠 예측

# results = model.predict(source='0', save=True, conf=0.25)  # 0은 기본 웹캠을 의미